In [9]:
# ### basic required packages ###
!pip install numpy
!pip install pyopenms
!pip install biopython
!pip install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.0/49.0 MB 6.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 7.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 1.6 MB/s eta 0:00:00a 0:00:01


In [10]:
# ### GOOGLE AUTH ###
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
!pip3 install --upgrade oauth2client 
!pip3 install --upgrade oauth2client 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 56.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.4/121.4 kB 204.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.3/182.3 kB 200.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.9/96.9 kB 98.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.9/230.9 kB 113.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 kB 157.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.9/83.9 kB 127.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 1.5 MB/s eta 0:00:00a 0:00:01


In [11]:
#######################
###  Запускать тут ####
#######################

import glob
import pandas as pd
import os

# # CHANGE THIS
# EXPERIMENT_NAME = "KJE0006"

# Список для хранения обработанных DataFrame из каждого файла


def xlsx_paths(EXPERIMENT_NAME):
    # specifying the path to csv files
    path = f"./requests/{EXPERIMENT_NAME}"

    # csv files in the path
    file_paths = glob.glob(path + "/*.xlsx")
    return file_paths

def csv_paths(EXPERIMENT_NAME):
    # specifying the path to csv files
    path = f"./requests/{EXPERIMENT_NAME}"

    # csv files in the path
    file_paths = glob.glob(path + "/*.csv")
    return file_paths

def is_valid_sequence(seq):

    if pd.isna(seq):
        return False
    valid_chars = set("ATGCUYRSWKMBDHVN.-")
    return all(c in valid_chars for c in str(seq))

def process_excel_files(xlsx_paths, dfs_list):

    for file_path in xlsx_paths:
        # Чтение файла Excel
        df = pd.read_excel(file_path, header=None)
        
        if len(df.columns)==1:
            df = df[0].str.split(',', expand=True)

        if len(df.columns)<4:
            return print("ERROR IN INPUT FILE")
        df.dropna(how='any', inplace=True)
        
        # df curation
        for column in df.columns:
            df[column] = df[column].apply(lambda x: unicodedata.normalize('NFC', str(x)))
        #  pandas' Series.str.strip() method
        df = df.apply(lambda x: x.str.strip())
        
        df[0] = df[0].apply(lambda x: x.upper())

        # Получение имени самого левого столбца
        leftmost_column = df.columns[0]
        print(file_path, len(df))
        # Фильтрация строк согласно заданным условиям
        df = df[df[leftmost_column].apply(is_valid_sequence)]
        
        print(file_path, len(df))
        # Добавление обработанного DataFrame в список
        dfs_list.append(df)
        
        if dfs_list != None:
            pass
        else:
            dfs_list = []
            print([])
        
    return dfs_list
        
    
def process_csv_files(csv_paths, dfs_list):

    for file_path in csv_paths:
        # Чтение файла Excel
        df = pd.read_csv(file_path, header=None, sep=",")
        print(df)
        df.dropna(how='any', inplace=True)
        if len(df.columns)==1:
            df = df[0].str.split(',', expand=True)

        if len(df.columns)<4:
            return print("ERROR IN INPUT FILE")

        # df curation
        for column in df.columns:
            df[column] = df[column].apply(lambda x: unicodedata.normalize('NFC', str(x)))
        #  pandas' Series.str.strip() method
        df = df.apply(lambda x: x.str.strip())
        
        df[0] = df[0].apply(lambda x: x.upper())

        # Получение имени самого левого столбца
        leftmost_column = df.columns[0]
        print(file_path, len(df))
        # Фильтрация строк согласно заданным условиям
        df = df[df[leftmost_column].apply(is_valid_sequence)]
        
        # Добавление обработанного DataFrame в список
        dfs_list.append(df)
        
        if dfs_list != None:
            pass
        else:
            dfs_list = []
            print([])
            
    return dfs_list
        
def process_files(EXPERIMENT_NAME):
    dfs_list = []
    dfs_list = process_excel_files(xlsx_paths(EXPERIMENT_NAME), dfs_list)
    if dfs_list != None:
        print(dfs_list)
    else:
        dfs_list = []
    dfs_list = process_csv_files(csv_paths(EXPERIMENT_NAME), dfs_list)
    return dfs_list

# EXPERIMENT_NAME = input()
# dfs_list=process_files(EXPERIMENT_NAME)



In [12]:
# # process_excel_files(xlsx_paths(EXPERIMENT_NAME), dfs_list)
# from google.colab import data_table
# data_table.enable_dataframe_formatter()

# from IPython.display import display

# # print the DataFrame
# display(dfs_list[-2][10:])

# Nikiteev_df=dfs_list[-2]

# Nikiteev_df=Nikiteev_df.dropna(how='any', inplace=True)

# display(Nikiteev_df[10:59])


In [13]:
def export_to_input_files(dfs_list, EXPERIMENT_NAME):
    # Конкатенация всех обработанных DataFrame по строкам
    final_df = None
    final_df = pd.concat(dfs_list, axis=0, join='outer', ignore_index=True)
    if (len(final_df)>384 and len(final_df)<=768):

        df_A = final_df[:384]
        df_B = final_df[384:]

        # Сохранение результата в новый файл CSV
        df_A.to_csv(f"./result_input/{EXPERIMENT_NAME}_concatenated_file_plate_A.csv", index=False, sep = ',', header = None, encoding = "UTF-8")
        # Сохранение результата в новый файл Excel
        df_A.to_excel(f"./result_input/{EXPERIMENT_NAME}_concatenated_file_plate_A.xlsx", index=False, header = None, encoding = "UTF-8")

         # Сохранение результата в новый файл CSV
        df_B.to_csv(f"./result_input/{EXPERIMENT_NAME}_concatenated_file_plate_B.csv", index=False, sep = ',', header = None, encoding = "UTF-8")
        # Сохранение результата в новый файл Excel
        df_B.to_excel(f"./result_input/{EXPERIMENT_NAME}_concatenated_file_plate_B.xlsx", index=False, header = None, encoding = "UTF-8")

    elif len(final_df)>768:
    
        print("error, too much oligos HALP")
    
    else:

        # Сохранение результата в новый файл CSV
        final_df.to_csv(f"./result_input/{EXPERIMENT_NAME}_concatenated_file.csv", index=False, sep = ',', header = None, encoding = "UTF-8")

        # Сохранение результата в новый файл Excel
        final_df.to_excel(f"./result_input/{EXPERIMENT_NAME}_concatenated_file.xlsx", index=False, header = None, encoding = "UTF-8")

In [14]:
#!/usr/bin/env python
#
# File: DrOligoInputPreparation_withFormsAPI.py
# Author: Oleg Fedorov, Kamil Zaynullin
#
# Copyright (C) 2023 Group of automated synthesis
#
# This file is part of gene synthesis automatization pipeline implemented in SBM.

from __future__ import print_function

import io
import os

########################
## GOOGLE API IMPORTS ##
########################
from apiclient import discovery
from google.oauth2.credentials import Credentials
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaIoBaseDownload
from httplib2 import Http
from oauth2client import client, file, tools

SCOPES = "https://www.googleapis.com/auth/drive"
DISCOVERY_DOC = "https://forms.googleapis.com/$discovery/rest?version=v1"


def creating_dirs(EXPERIMENT_NAME):
    #################
    ## CREATE DIR  ##
    #################
    cwd = os.getcwd()
    dir = os.path.join(cwd, "requests", EXPERIMENT_NAME)
    if not os.path.exists(dir):
        os.mkdir(dir)


def google_auth():
    #################
    ## GOOGLE AUTH ##
    #################

    """
    caution! it appeares to be deprecated!!!

    """
    cwd = os.getcwd()
    token_file = os.path.join(cwd, "token.json")
    if os.path.exists(token_file):
        print("auth token already exists")
        store = file.Storage(token_file)
        creds = store.get()
    else:
        store = file.Storage("token.json")
        creds = None
        if not creds:
            flow = client.flow_from_clientsecrets("credentials.json", SCOPES)

            creds = tools.run_flow(flow, store)
    return creds


def retrive_forms_data(creds):
    ##################
    ## GOOGLE FORMS ##
    ##################
    service = discovery.build(
        "forms",
        "v1",
        http=creds.authorize(Http()),
        discoveryServiceUrl=DISCOVERY_DOC,
        static_discovery=False,
    )

    form_id = "1bVGyD-pz5w3BXM48yHWnhEDVnYNVgzRrF3ysvjAbqHQ"
    result = service.forms().responses().list(formId=form_id).execute()
    file_list = [
        response["answers"]["778e4a4f"]["fileUploadAnswers"]["answers"]
        for response in result["responses"]
    ]
    print(file_list)
    return file_list


####################################################
# downloading from Google Disk and writing to file #
####################################################
def download_request_file(fileID, filename, creds, EXPERIMENT_NAME):
    try:
        service = discovery.build("drive", "v3", credentials=creds)

        file_id = fileID

        # pylint: disable=maybe-no-member
        request = service.files().get_media(fileId=file_id)
        file = io.BytesIO()
        downloader = MediaIoBaseDownload(file, request)
        done = False
        while done is False:
            status, done = downloader.next_chunk()
            print(f"Download {int(status.progress() * 100)}.")

    except HttpError as error:
        print(f"An error occurred: {error}")
        file = None
    filepath = os.path.join(os.getcwd(), "requests", EXPERIMENT_NAME, filename)
    with open(filepath, "wb") as outfile:
        # Copy the BytesIO stream to the output file
        outfile.write(file.getbuffer())

    return print(file.getvalue())


def download_all_files(creds, EXPERIMENT_NAME):
    for file in retrive_forms_data(creds):
        download_request_file(file[0]["fileId"], file[0]["fileName"], creds, EXPERIMENT_NAME)


##########
## main ##
##########
def main() -> None:
    EXPERIMENT_NAME = input("enter EXPERIMENT_NAME:")
    creating_dirs(EXPERIMENT_NAME)
    # retrive_forms_data()
    creds = google_auth()
    download_all_files(creds, EXPERIMENT_NAME)
    print("downloads finished")
    return EXPERIMENT_NAME

In [19]:
#####################################
#### EXPORTING EMAILS FROM FORMS ####
#####################################

import re  
import json

def get_emails_data():
    creds = google_auth()

    service = discovery.build(
        "forms",
        "v1",
        http=creds.authorize(Http()),
        discoveryServiceUrl=DISCOVERY_DOC,
        static_discovery=False,
    )

    form_id = "1bVGyD-pz5w3BXM48yHWnhEDVnYNVgzRrF3ysvjAbqHQ"
    result = service.forms().responses().list(formId=form_id).execute()
#     print(json.dumps(result, indent=2))
    
    return result

def remove_duplicate_emails(response_str):
    response_dict = json.loads(response_str) # Преобразование JSON-строки в Python-объект
    email_set = set()

    for response in response_dict['responses']:
        email = response.get('respondentEmail', None)
        if email:
            email_set.add(email)

    email_list = list(email_set)
    return email_list

regex = re.compile(r'([A-Za-z0-9]+[.-_])*[A-Za-z0-9\-]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+')  
  
def emailValid(unique_emails):
    for item in unique_emails:
        if re.fullmatch(regex, item):
            print(item, "The given mail is valid")  
        else:  
            print(item, "The given mail is invalid")  
            unique_emails.remove(item)
    return unique_emails

def create_file_with_emails():
    # Использования функции
    result = get_emails_data()
    response_str = json.dumps(result, indent=2)
    unique_emails = remove_duplicate_emails(response_str)
    unique_emails = emailValid(unique_emails)

    # write to file
    with open('unique_emails.txt', 'w') as f:
        for email in unique_emails:
            f.write(f"{email}\n")


In [20]:
create_file_with_emails()

auth token already exists
herr.romanoff@gmail.com The given mail is valid
kuzminajk2000@gmail.com The given mail is valid
thejuzzeppe@gmail.com The given mail is valid
olegteravolt@gmail.com The given mail is valid


In [ ]:
def combined_pipeline():
    EXPERIMENT_NAME = main()
    print(EXPERIMENT_NAME)
    dfs_list=process_files(EXPERIMENT_NAME)
    export_to_input_files(dfs_list, EXPERIMENT_NAME)
combined_pipeline()

In [37]:
unique_emails

{'herr.romanoff@gmail.com',
 'kuzmina-je@sysbiomed.ru',
 'kuzmina_je@sysbiomedl.ru',
 'kuzminajk2000@gmail.com',
 'olegteravolt@gmail.com',
 'orcfire658@gmail.com',
 'razor1710@mail.ru',
 'thejuzzeppe@gmail.com',
 'тест'}

orcfire658@gmail.com The given mail is valid
thejuzzeppe@gmail.com The given mail is valid
kuzminajk2000@gmail.com The given mail is valid
kuzmina-je@sysbiomed.ru The given mail is valid
тест The given mail is invalid
herr.romanoff@gmail.com The given mail is valid
olegteravolt@gmail.com The given mail is valid
kuzmina_je@sysbiomedl.ru The given mail is valid
razor1710@mail.ru The given mail is valid


In [28]:
# !jupyter labextension install @jupyterlab/dataregistry-extension